<a href="https://colab.research.google.com/github/jithesh9539/Deep_Learning_Assignment1/blob/main/Blackbox_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>